In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score as auc

In [11]:
train_tag = pd.read_csv('./b/train_tag_without_Enc.csv')

test_tag = pd.read_csv('./b/test_tag_without_Enc.csv')

In [3]:
train_trd = pd.read_csv('./b/train_trd_max_features.csv')
test_trd = pd.read_csv('./b/test_trd_max_features.csv')


In [5]:

data_train_tag = pd.read_csv('./训练数据集1/训练数据集_tag.csv')
data_train_trd = pd.read_csv('./训练数据集1/训练数据集_trd.csv')
data_train_beh = pd.read_csv('./训练数据集1/训练数据集_beh.csv')

data_test_tag = pd.read_csv('./b/评分数据集_tag_b.csv')
data_test_trd = pd.read_csv('./b/评分数据集_trd_b.csv')
data_test_beh = pd.read_csv('./b/评分数据集_beh_b.csv')


In [6]:
# beh训练集
user_pgTimes = pd.DataFrame(data_train_beh.id.value_counts())
user_pgTimes['index'] = user_pgTimes.index
user_pgTimes.index = range(len(user_pgTimes))
user_pgTimes.columns = ['pg_times', 'user']
user_pgTimes = user_pgTimes[['user', 'pg_times']]

In [8]:
# beh测试集
test_user_pgTimes = pd.DataFrame(data_test_beh.id.value_counts())
test_user_pgTimes['index'] = test_user_pgTimes.index
test_user_pgTimes.index = range(len(test_user_pgTimes))
test_user_pgTimes.columns = ['pg_times', 'user']
test_user_pgTimes = test_user_pgTimes[['user', 'pg_times']]
test_user_pgTimes.shape

(1232, 2)

### one-hot 编码

In [12]:
categorical_cols = train_tag.select_dtypes(include='object').columns
categorical_cols

Index(['id', 'gdr_cd', 'mrg_situ_cd', 'edu_deg_cd', 'acdm_deg_cd'], dtype='object')

In [18]:
del(data)

In [19]:
data = pd.concat([train_tag, test_tag], ignore_index=True, sort=False)

In [20]:
categorical_cols = data.select_dtypes(include='object').columns
categorical_cols

Index(['id', 'gdr_cd', 'mrg_situ_cd', 'edu_deg_cd', 'acdm_deg_cd'], dtype='object')

In [29]:
one_hot_features = data.loc[:, ['id', 'edu_deg_cd', 'acdm_deg_cd']]

In [30]:
edu_ohe_2 = pd.get_dummies(one_hot_features.loc[:, ['edu_deg_cd', 'acdm_deg_cd']])

In [32]:
edu_ohe_2['id'] = one_hot_features['id']

In [34]:
one_hot_features = pd.merge(one_hot_features, edu_ohe_2, how='left', left_on='id', right_on='id')

In [35]:
one_hot_features

,id,edu_deg_cd,acdm_deg_cd,edu_deg_cd_A,edu_deg_cd_B,edu_deg_cd_C,edu_deg_cd_D,edu_deg_cd_F,edu_deg_cd_G,edu_deg_cd_J,...,edu_deg_cd_M,edu_deg_cd_Z,edu_deg_cd_~,acdm_deg_cd_30,acdm_deg_cd_31,acdm_deg_cd_C,acdm_deg_cd_D,acdm_deg_cd_F,acdm_deg_cd_G,acdm_deg_cd_Z
0,U7A4BAD,B,Z,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,U557810,F,G,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,U1E9240,A,C,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,U6DED00,F,Z,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,UDA8E28,B,Z,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43918,U0FBFE2,B,30,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
43919,U506AD2,C,31,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
43920,U287B33,Z,Z,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
43921,U15134F,B,30,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [36]:
one_hot_features.drop([ 'edu_deg_cd', 'acdm_deg_cd'], axis=1, inplace=True)

In [37]:
data.drop([ 'edu_deg_cd', 'acdm_deg_cd'], axis=1, inplace=True)


TypeError: merge() got multiple values for argument 'how'

In [39]:
data = pd.merge(data, one_hot_features, how='left', left_on='id', right_on='id')

In [41]:
train_tag = data.iloc[:len(train_tag),:]
test_tag = data.iloc[len(train_tag):, :]
test_tag.index = np.arange(len(test_tag))
test_tag.drop(['flag'], axis=1, inplace=True)

/home/esterl/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## 合并上述表格

In [42]:
train_data = pd.merge(train_tag,user_pgTimes, how='left', left_on='id', right_on='user')
train_data = pd.merge(train_data, train_trd, how='left', left_on='id', right_on='user')
train_data.head()

,id,flag,gdr_cd,age,mrg_situ_cd,job_year,ic_ind,fr_or_sh_ind,dnl_mbl_bnk_ind,dnl_bind_cmb_lif_ind,...,B_out,C_in,out_count,in_count,cny_trx_amt,Dat_Flg3_Cd,Trx_Cod1_Cd,is_in_first_10days,is_in_middle_10days,is_in_last_10days
0,U7A4BAD,0.0,M,41,B,3,0,0,0,1,...,1.0,2.0,15000.00,17400.00,10000.0,C,2.0,1.0,1.0,1.0
1,U557810,0.0,M,35,A,4,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U1E9240,0.0,M,53,B,9,0,0,1,1,...,6.0,11.0,131158.00,105718.04,68000.0,A,2.0,7.0,3.0,7.0
3,U6DED00,0.0,F,41,B,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,UDA8E28,0.0,F,42,B,3,1,1,1,0,...,5.0,13.0,70416.04,79505.55,30000.0,A,2.0,7.0,4.0,7.0


In [43]:
train_data.drop(['user_x', 'user_y'], axis=1, inplace=True)

train_data.shape

(39923, 83)

In [44]:
test_data = pd.merge(test_tag, test_user_pgTimes, how='left', left_on='id', right_on='user')
test_data = pd.merge(test_data, test_trd, how='left', left_on='id', right_on='user')
test_data.drop(['user_x', 'user_y'], axis=1, inplace=True)
test_data.shape

(4000, 82)

In [45]:
# pgtimes用众数填充
train_data.pg_times.fillna(train_data.pg_times.mode()[0], inplace=True)
test_data.pg_times.fillna(train_data.pg_times.mode()[0], inplace=True)